In [1]:
import sys
import os
import pandas as pd
from itertools import product

notebook_dir = os.path.dirname(os.path.realpath("__file__"))
project_root_dir = os.path.join(notebook_dir, '..')
if project_root_dir not in sys.path:
    sys.path.append(project_root_dir)

from logic.mortgage_repayments import compute_monthly_mortgage_repayments
from logic.income_calculations import compute_income_after_essentials, compute_income_after_total_taxation
from logic.paye_taxation import compute_student_loan_repayments

In [2]:
annual_pension_contributions_rate = 0.05

annual_council_tax_contribution = 1_475 / 2
annual_home_insrance_contribution = 200 / 2
annual_ground_rent_contribution = 50 / 2
annual_service_charge_contribution = 2500 / 2

annual_minimum_additional_living_costs = 8_500
annual_inflated_additional_minimum_additional_living_costs = 13_000

mortgage_length_in_years = 25

In [3]:
def compute_monthly_income_left_for_mortgage_per_month(
    salary,
    pension_contributions_rate,
    annual_council_tax,
    annual_home_insurance,
    annual_ground_rent,
    annual_service_charge,
    lifestyle,
    annual_additional_minimum_living_costs,
    annual_inflated_additional_minimum_living_costs
):
    income_after_total_taxation = compute_income_after_total_taxation(salary, pension_contributions_rate, annual_council_tax)
    annual_additional_housing_costs = annual_home_insurance + annual_ground_rent + annual_service_charge

    if lifestyle == "45p_net_on_mortgage":
        income_left_for_mortgage = 0.45 * income_after_total_taxation
    elif lifestyle == "35p_net_on_mortgage":
        income_left_for_mortgage = 0.35 * income_after_total_taxation
    elif lifestyle == "28p_net_on_housing":
        income_left_for_housing = 0.28 * income_after_total_taxation
        income_left_for_mortgage = income_left_for_housing - annual_home_insurance - annual_ground_rent - annual_service_charge
    elif lifestyle == "25p_net_on_mortgage":
        income_left_for_mortgage = 0.25 * income_after_total_taxation
    elif lifestyle == "25p_gross_on_debts":
        income_left_for_mortgage = 0.25 * salary - compute_student_loan_repayments(salary)
    elif lifestyle == "50-30-20":
        income_left_for_mortgage = income_after_total_taxation*0.5 - annual_additional_housing_costs - annual_additional_minimum_living_costs
    elif lifestyle == "inflated_75-25":
        income_left_for_mortgage = income_after_total_taxation*0.75 - annual_additional_housing_costs - annual_inflated_additional_minimum_living_costs
    elif lifestyle == "current_essentials":
        income_left_for_mortgage = compute_income_after_essentials(
            salary,
            pension_contributions_rate,
            annual_council_tax,
            annual_home_insurance,
            annual_ground_rent,
            annual_service_charge,
            annual_additional_minimum_living_costs
        )
    elif lifestyle == "inflated_essentials":
        income_left_for_mortgage = compute_income_after_essentials(
            salary,
            pension_contributions_rate,
            annual_council_tax,
            annual_home_insurance,
            annual_ground_rent,
            annual_service_charge,
            annual_inflated_additional_minimum_living_costs
        )
    else:
        raise Exception(f"Unsupported lifestyle: {lifestyle}")

    income_left_for_mortgage_per_month = income_left_for_mortgage / 12
    return income_left_for_mortgage_per_month

In [4]:
def compute_highest_interest_rate_affordable(
    salary,
    pension_contributions_rate,
    fraction_of_mortgage_repayments,
    annual_council_tax,
    annual_home_insurance,
    annual_ground_rent,
    annual_service_charge,
    total_mortgage_size,
    total_mortgage_length_in_years,
    lifestyle,
    annual_additional_minimum_living_costs,
    annual_inflated_additional_minimum_living_costs
):
    income_left_for_mortgage_per_month = compute_monthly_income_left_for_mortgage_per_month(
        salary,
        pension_contributions_rate,
        annual_council_tax,
        annual_home_insurance,
        annual_ground_rent,
        annual_service_charge,
        lifestyle,
        annual_additional_minimum_living_costs,
        annual_inflated_additional_minimum_living_costs
    )
    
    starting_interest_rate = 0.0025
    interest_rate = starting_interest_rate
    interest_rate_incrementation = 0.0025

    # LOL
    while True:
        monthly_mortgage_repayments = compute_monthly_mortgage_repayments(
            total_mortgage_size, total_mortgage_length_in_years, interest_rate
        )
        monthly_mortgage_repayments = monthly_mortgage_repayments * fraction_of_mortgage_repayments

        if monthly_mortgage_repayments > income_left_for_mortgage_per_month:
            if interest_rate == starting_interest_rate:
                return "N/A"
            return 100*(interest_rate - interest_rate_incrementation)

        interest_rate = interest_rate + interest_rate_incrementation
    

In [5]:
def compute_pivot_table(
    df,
    pivot_row,
    pivot_columns,
    annual_council_tax,
    annual_home_insrance,
    annual_ground_rent,
    annual_service_charge,
    mortgage_length_in_years,
    print_assumptions=True
):
    if print_assumptions:
        print('Assumptions: (only my payments)')
        print(f'- Mortgage length in years: {mortgage_length_in_years}')
        print(f'- Annual council tax: {annual_council_tax}')
        print(f'- Annual home insurance: {annual_home_insrance}')
        print(f'- Annual ground rent: {annual_ground_rent}')
        print(f'- Annual service charge: {annual_service_charge}')

    return df.pivot(index=pivot_row, columns=pivot_columns)

In [6]:

salaries = [salary for salary in range(60_000, 100_001, 5_000)]
mortgage_sizes = [mortgage_size for mortgage_size in range(250_000, 260_001, 10_000)]
lifestyles = ["50-30-20", "35p_net_on_mortgage", "inflated_essentials"]

combinations = list(product(salaries, mortgage_sizes, lifestyles))

df = pd.DataFrame(combinations, columns=['salary', 'mortgage_size', 'lifestyle'])

df["max_interest_rate_affordable"] = df.apply(lambda x: compute_highest_interest_rate_affordable(
    x["salary"],
    annual_pension_contributions_rate,
    0.5,
    annual_council_tax_contribution,
    annual_home_insrance_contribution,
    annual_ground_rent_contribution,
    annual_service_charge_contribution,
    x["mortgage_size"],
    mortgage_length_in_years,
    x["lifestyle"],
    annual_minimum_additional_living_costs,
    annual_inflated_additional_minimum_additional_living_costs
), axis=1)

df = compute_pivot_table(
    df,
    "salary",
    ["mortgage_size", "lifestyle"],
    annual_council_tax_contribution,
    annual_home_insrance_contribution,
    annual_ground_rent_contribution,
    annual_service_charge_contribution,
    mortgage_length_in_years
)

df = df.astype(float).round(2)

df

Assumptions: (only my payments)
- Mortgage length in years: 25
- Annual council tax: 737.5
- Annual home insurance: 100.0
- Annual ground rent: 25.0
- Annual service charge: 1250.0


max_interest_rate_affordable                      \
mortgage_size                       250000                       
lifestyle                         50-30-20 35p_net_on_mortgage   
salary                                                           
60000                                 6.25               10.25   
65000                                 7.50               11.00   
70000                                 8.75               11.75   
75000                                 9.75               12.50   
80000                                10.75               13.00   
85000                                12.00               13.75   
90000                                13.00               14.50   
95000                                14.00               15.25   
100000                               15.00               16.00   

                                                                \
mortgage_size                       260000                       
lifestyle     inflated_essentials 50-30-20 35p_net_on_mortgage   
salary                                                           
60000                       20.25     6.00                9.75   
65000                       22.00     7.00               10.50   
70000                       24.00     8.25               11.00   
75000                       25.75     9.25               11.75   
80000                       27.50    10.25               12.50   
85000                       29.50    11.25               13.25   
90000                       31.25    12.25               14.00   
95000                       33.25    13.25               14.50   
100000                      35.00    14.25               15.25   

                                   
mortgage_size                      
lifestyle     inflated_essentials  
salary                             
60000                       19.25  
65000                       21.25  
70000                       23.00  
75000                       24.75  
80000                       26.50  
85000                       28.25  
90000                       30.00  
95000                       31.75  
100000                      33.75